Step 1: Import necessary packages

In [2]:
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch


Could not import packages for CMIknn and GPDC estimation
Could not import packages for CMIknn and GPDC estimation


In [2]:
def normalize_0_mean_1_std(inp_series):
    inp_series=inp_series.copy()
    mean_ts=np.array([inp_series.mean(axis=1)]).transpose()
    mean_ts_mtrx = mean_ts*np.ones((1,inp_series.shape[1]));
    unb_data_mtrx = inp_series - mean_ts_mtrx
    p = np.power(unb_data_mtrx,2)
    s=np.array([p.sum(axis=1)]).transpose()
    sc=np.sqrt(s/p.shape[1])
    sc2=sc*(np.ones((1,p.shape[1])))
    nrm= np.divide(unb_data_mtrx,sc2)
    return nrm

step 2: Get data from the file and encode the labels using LabelEncoder class.

In [3]:
def multivariate_split(X,ar_order, valid_percent=0):
    X=X.copy()
    TS=np.shape(X)[1]
    n_vars=np.shape(X)[0]
    val_num=int(valid_percent*TS)
    my_data_train=torch.zeros((TS-ar_order-val_num,ar_order,n_vars))
    my_data_y_train=torch.zeros((TS-ar_order-val_num,1,n_vars))
    my_data_val=torch.zeros((val_num,ar_order,n_vars))
    my_data_y_val=torch.zeros((val_num,1,n_vars))
    for i in range(TS-ar_order-val_num):
        my_data_train[i]=torch.from_numpy(X.transpose()[i:i+ar_order,:])
        my_data_y_train[i]=torch.from_numpy(X.transpose()[i+ar_order,:])

    for i in range(TS-ar_order-val_num, TS-ar_order,1):
        my_data_val[i-(TS-ar_order-val_num)]=torch.from_numpy(X.transpose()[i:i+ar_order,:])
        my_data_y_val[i-(TS-ar_order-val_num)]=torch.from_numpy(X.transpose()[i+ar_order,:])
    return my_data_train, my_data_y_train, my_data_val, my_data_y_val

In [4]:
def calc_f_stat(RSS_R,RSS_U,n,pu,pr):
    f_GC = ((RSS_R-RSS_U)/(RSS_U))*((n-pu-1)/(pu-pr));
    return f_GC

Now I am using in for Granger causality

Step 1: Load data

In [3]:
from scipy.io import loadmat

ar_order=2


in_data_name='datasets/7SYNTHETICS/logistic_1.mat'

GroundTruth=[loadmat(in_data_name)['Adj'] for i in range(50)]
ts_logistic=[loadmat(in_data_name)['pt_N'][i,:,-500:] for i in range(50)]

Step 2: Normalize data

In [6]:
X_normalized=normalize_0_mean_1_std(inp_series=ts_logistic[0])

Step 3: Split the data into train and test

In [7]:
X_train, Y_train , X_test, Y_test=multivariate_split(X=X_normalized,ar_order=ar_order)

X_train=torch.flatten(X_train, start_dim=1)
X_test=torch.flatten(X_test, start_dim=1)

Y_train=torch.flatten(Y_train, start_dim=1)
Y_test=torch.flatten(Y_test, start_dim=1)

print(X_train.shape)

torch.Size([498, 4])


 Step 4: Obtain $k_f$ number of cluster centers with k-means clustering. It has dimensions of $ \in k_f \times Nd$

In [8]:
k_f=3
km= KMeans(n_clusters= k_f, max_iter= 100, random_state=123)
km.fit(X_train)
cent= km.cluster_centers_

print(cent.shape)

(3, 4)


Step 5: Set the width of the kernel as the mean distance between cluster centers. (I select the max)

In [9]:
max=0 

for i in range(k_f):
    for j in range(k_f):
        d= np.linalg.norm(cent[i]-cent[j])
        if(d> max):
            max= d
d= max

sigma= d/math.sqrt(2*k_f)
print(sigma)

1.2204506596243991


In [10]:
# maxg=0 

# for ii in range(k_g):
#     for jj in range(k_g):
#         dg= np.linalg.norm(cent_W_s[ii]-cent_W_s[jj])
#         if(dg> max):
#             maxg= dg
# dg= maxg

# sigmag= dg/math.sqrt(2*k_g)
# print(sigma)

Step 6: Iterate through all $N$ time-series from 1 to $n$ where $n \in N$

In [11]:
sig_d=np.zeros((np.shape(X_normalized)[0],np.shape(X_normalized)[0]));
sig=np.zeros((np.shape(X_normalized)[0],np.shape(X_normalized)[0]));




for i in range(X_normalized.shape[0]):
    Z_temp=X_normalized.copy()
    Z_train, Z_train_label , _ , _=multivariate_split(X=Z_temp,ar_order=ar_order)
    Z_train=torch.flatten(Z_train, start_dim=1)
    Z_train_label=torch.flatten(Z_train_label, start_dim=1)
    
    # Obtain phase space Z_s by exclusing time series of of x_s
    Z_s_train, Z_s_train_label , _ , _=multivariate_split(X=np.delete(Z_temp,[i],axis=0),ar_order=ar_order)
    # Obtain phase space reconstruction of x_s
    W_s_train, W_s_train_label , _ , _=multivariate_split(X=np.array([Z_temp[i]]),ar_order=ar_order)
    
    # Flatten data
    Z_s_train=torch.flatten(Z_s_train, start_dim=1)
    Z_s_train_label=torch.flatten(Z_s_train_label, start_dim=1)

    W_s_train=torch.flatten(W_s_train, start_dim=1)
    W_s_train_label=torch.flatten(W_s_train_label, start_dim=1)
    # Obtain k_g number of cluster centers in the phase space W_s with k-means clustering, will have dim=(k_g * d)
    k_g=2
    kmg= KMeans(n_clusters= k_g, max_iter= 100, random_state=123)
    kmg.fit(W_s_train)
    cent_W_s= kmg.cluster_centers_
    # Calculate activations for each of the k_g neurons
    shape= W_s_train.shape
    row= shape[0]
    column= k_g
    G= np.empty((row,column), dtype= float)
    maxg=0 

    for ii in range(k_g):
        for jj in range(k_g):
            dg= np.linalg.norm(cent_W_s[ii]-cent_W_s[jj])
            if(dg> maxg):
                maxg= dg
    dg= maxg

    sigmag= dg/math.sqrt(2*k_g)
    for ii in range(row):
        for jj in range(column):
            dist= np.linalg.norm(W_s_train[ii]-cent_W_s[jj])
            G[ii][jj]= math.exp(-math.pow(dist,2)/math.pow(2*sigmag,2))
    # Generalized radial basis function
    g_ws=np.array([G[ii]/sum(G[ii]) for ii in range(len(G))])
    # Calculate activations for each of the k_f neurons 
    shape= Z_s_train.shape
    row= shape[0]
    column= k_f
    F= np.empty((row,column), dtype= float)
    for ii in range(row):
        for jj in range(column):
            cent_temp=cent.copy()
            cent_temp=np.delete(cent_temp,np.arange(jj,jj+ar_order),axis=1)
            dist= np.linalg.norm(Z_s_train[ii]-cent_temp)
            F[ii][jj]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))
    # Generalized radial basis function
    f_zs=np.array([F[ii]/sum(F[ii]) for ii in range(len(F))])
    
    # Prediction in the presence of x_s
    dims=np.max([k_f,k_g])
    num_samples=f_zs.shape[0]
    f_new=np.ones((num_samples,dims))
    g_new=np.ones((num_samples,dims))

    for ii in range(f_new.shape[0]):
        f_new[ii,:f_zs.shape[1]]=f_zs[ii,:]
        g_new[ii,:g_ws.shape[1]]=g_ws[ii,:]
    fg_combined=np.concatenate((g_new,f_new),axis=1)

    GTG= np.dot(fg_combined.T,fg_combined)
    GTG_inv= np.linalg.inv(GTG)
    fac= np.dot(GTG_inv,fg_combined.T)
    W_presence= np.dot(fac,Z_train_label)

    prediction_presence= np.dot(fg_combined,W_presence)
    error_presence=prediction_presence-np.array(Z_train_label)
#     dims=k_f+k_g
#     num_samples=f_zs.shape[0]
#     f_new=np.zeros((num_samples,dims))

#     for ii in range(f_new.shape[0]):
#         f_new[ii,:k_f]=f_zs[ii,:]
#         f_new[ii,k_f:]=g_ws[ii,:]

#     GTG= np.dot(f_new.T,f_new)
#     GTG_inv= np.linalg.inv(GTG)
#     fac= np.dot(GTG_inv,f_new.T)
#     W_presence= np.dot(fac,Z_train_label)

#     prediction_presence= np.dot(f_new,W_presence)
#     error_presence=prediction_presence-np.array(Z_train_label)
    sig[i,:]=np.diag(np.cov(error_presence.T))
    
    # Prediction without x_s
    GTG= np.dot(f_zs.T,f_zs)
    GTG_inv= np.linalg.inv(GTG)
    fac= np.dot(GTG_inv,f_zs.T)
    W_absence= np.dot(fac,Z_train_label)

    prediction_absence= np.dot(f_zs,W_absence)
    error_absence=prediction_absence-np.array(Z_train_label)
    sig_d[i,:]=np.diag(np.cov(error_absence.T))
# Comupte the Granger causality index

Aff=np.log(np.divide(sig_d,sig))
Aff=(Aff>0)*Aff
np.fill_diagonal(Aff,0)
Aff

f_stat=calc_f_stat(sig_d, sig, n=num_samples+1, pu=k_f+k_g, pr=k_f)

In [74]:
f_new=np.concatenate((f_zs,g_ws),axis=1)

GTG= np.dot(f_new.T,f_new)
GTG_inv= np.linalg.inv(GTG)
fac= np.dot(GTG_inv,f_new.T)
W_presence= np.dot(fac,Z_train_label)

prediction_presence= np.dot(f_new,W_presence)
error_presence=prediction_presence-np.array(Z_train_label)

In [73]:
W_presence.shape

(5, 2)

In [63]:
A1=np.concatenate((f_zs, np.zeros_like(g_ws)),axis=1)
A2=np.concatenate((np.zeros_like(f_zs), g_ws),axis=1)
AA=np.concatenate((A1,A2),axis=0)
fg_new=np.concatenate((f_zs,g_ws),axis=1)

In [68]:
AA=np.concatenate((A1,A2),axis=0)

In [12]:
np.fill_diagonal(f_stat,0)
f_stat>0.05

array([[False,  True],
       [False, False]])

In [13]:
(Aff>0.00)*1.

array([[0., 1.],
       [0., 0.]])

In [14]:
GroundTruth[0]

array([[0, 1],
       [0, 0]], dtype=uint8)

In [15]:
def multivariate_split(X,ar_order, valid_percent=0):
    X=X.copy()
    TS=np.shape(X)[1]
    n_vars=np.shape(X)[0]
    val_num=int(valid_percent*TS)
    my_data_train=torch.zeros((TS-ar_order-val_num,ar_order,n_vars))
    my_data_y_train=torch.zeros((TS-ar_order-val_num,1,n_vars))
    my_data_val=torch.zeros((val_num,ar_order,n_vars))
    my_data_y_val=torch.zeros((val_num,1,n_vars))
    for i in range(TS-ar_order-val_num):
        my_data_train[i]=torch.from_numpy(X.transpose()[i:i+ar_order,:])
        my_data_y_train[i]=torch.from_numpy(X.transpose()[i+ar_order,:])

    for i in range(TS-ar_order-val_num, TS-ar_order,1):
        my_data_val[i-(TS-ar_order-val_num)]=torch.from_numpy(X.transpose()[i:i+ar_order,:])
        my_data_y_val[i-(TS-ar_order-val_num)]=torch.from_numpy(X.transpose()[i+ar_order,:])
    return my_data_train, my_data_y_train, my_data_val, my_data_y_val


def calc_f_stat(RSS_R,RSS_U,n,pu,pr):
    f_GC = ((RSS_R-RSS_U)/(RSS_U))*((n-pu-1)/(pu-pr));
    return f_GC

def lsNGC(inp_series, ar_order=1, k_f=3, k_g=2, normalize=1):
    if normalize:
        X_normalized=normalize_0_mean_1_std(inp_series)

    km= KMeans(n_clusters= k_f, max_iter= 100, random_state=123)
    km.fit(X_train)
    cent= km.cluster_centers_


    max=0 

    for i in range(k_f):
        for j in range(k_f):
            d= np.linalg.norm(cent[i]-cent[j])
            if(d> max):
                max= d
    d= max

    sigma= d/math.sqrt(2*k_f)

    sig_d=np.zeros((np.shape(X_normalized)[0],np.shape(X_normalized)[0]));
    sig=np.zeros((np.shape(X_normalized)[0],np.shape(X_normalized)[0]));


    for i in range(X_normalized.shape[0]):
        Z_temp=X_normalized.copy()
        Z_train, Z_train_label , _ , _=multivariate_split(X=Z_temp,ar_order=ar_order)
        Z_train=torch.flatten(Z_train, start_dim=1)
        Z_train_label=torch.flatten(Z_train_label, start_dim=1)

        # Obtain phase space Z_s by exclusing time series of of x_s
        Z_s_train, Z_s_train_label , _ , _=multivariate_split(X=np.delete(Z_temp,[i],axis=0),ar_order=ar_order)
        # Obtain phase space reconstruction of x_s
        W_s_train, W_s_train_label , _ , _=multivariate_split(X=np.array([Z_temp[i]]),ar_order=ar_order)

        # Flatten data
        Z_s_train=torch.flatten(Z_s_train, start_dim=1)
        Z_s_train_label=torch.flatten(Z_s_train_label, start_dim=1)

        W_s_train=torch.flatten(W_s_train, start_dim=1)
        W_s_train_label=torch.flatten(W_s_train_label, start_dim=1)
        # Obtain k_g number of cluster centers in the phase space W_s with k-means clustering, will have dim=(k_g * d)
        kmg= KMeans(n_clusters= k_g, max_iter= 100, random_state=123)
        kmg.fit(W_s_train)
        cent_W_s= kmg.cluster_centers_
        # Calculate activations for each of the k_g neurons
        shape= W_s_train.shape
        row= shape[0]
        column= k_g
        G= np.empty((row,column), dtype= float)
        maxg=0 

        for ii in range(k_g):
            for jj in range(k_g):
                dg= np.linalg.norm(cent_W_s[ii]-cent_W_s[jj])
                if(dg> maxg):
                    maxg= dg
        dg= maxg

        sigmag= dg/math.sqrt(2*k_g)
        for ii in range(row):
            for jj in range(column):
                dist= np.linalg.norm(W_s_train[ii]-cent_W_s[jj])
                G[ii][jj]= math.exp(-math.pow(dist,2)/math.pow(2*sigmag,2))
        # Generalized radial basis function
        g_ws=np.array([G[ii]/sum(G[ii]) for ii in range(len(G))])
        # Calculate activations for each of the k_f neurons 
        shape= Z_s_train.shape
        row= shape[0]
        column= k_f
        F= np.empty((row,column), dtype= float)
        for ii in range(row):
            for jj in range(column):
                cent_temp=cent.copy()
                cent_temp=np.delete(cent_temp,np.arange(jj,jj+ar_order),axis=1)
                dist= np.linalg.norm(Z_s_train[ii]-cent_temp)
                F[ii][jj]= math.exp(-math.pow(dist,2)/math.pow(2*sigma,2))
        # Generalized radial basis function
        f_zs=np.array([F[ii]/sum(F[ii]) for ii in range(len(F))])

        # Prediction in the presence of x_s
        dims=np.max([k_f,k_g])
        num_samples=f_zs.shape[0]
        f_new=np.ones((num_samples,dims))
        g_new=np.ones((num_samples,dims))

        for ii in range(f_new.shape[0]):
            f_new[ii,:f_zs.shape[1]]=f_zs[ii,:]
            g_new[ii,:g_ws.shape[1]]=g_ws[ii,:]
        fg_combined=np.concatenate((g_new,f_new),axis=1)

        GTG= np.dot(fg_combined.T,fg_combined)
        GTG_inv= np.linalg.inv(GTG)
        fac= np.dot(GTG_inv,fg_combined.T)
        W_presence= np.dot(fac,Z_train_label)

        prediction_presence= np.dot(fg_combined,W_presence)
        error_presence=prediction_presence-np.array(Z_train_label)
        sig[i,:]=np.diag(np.cov(error_presence.T))

        # Prediction without x_s
        GTG= np.dot(f_zs.T,f_zs)
        GTG_inv= np.linalg.inv(GTG)
        fac= np.dot(GTG_inv,f_zs.T)
        W_absence= np.dot(fac,Z_train_label)

        prediction_absence= np.dot(f_zs,W_absence)
        error_absence=prediction_absence-np.array(Z_train_label)
        sig_d[i,:]=np.diag(np.cov(error_absence.T))
    # Comupte the Granger causality index

    Aff=np.log(np.divide(sig_d,sig))
    Aff=(Aff>0)*Aff
    np.fill_diagonal(Aff,0)
    f_stat=calc_f_stat(sig_d, sig, n=num_samples+1, pu=k_f+k_g, pr=k_f)
    np.fill_diagonal(f_stat,0)
    
    return Aff, f_stat